# S3 based progress tracker

A simple tracker that can be used to track the progress of a long process. For example, you are trying to download data from 2000-01-01 to 2020-01-01, and you split them into 20 years chunks, and you download them one by one. You can use tracker to track the last succeeded year.

In [10]:
import json
import dataclasses
from boto_session_manager import BotoSesManager

from fixa.aws.aws_s3_tracker import BaseTracker, Backend

In [3]:
# BaseTracker is a dataclasses, you have to use dataclasses.dataclass decorator here
@dataclasses.dataclass
class Tracker(BaseTracker):
    year: int = dataclasses.field()

    @classmethod
    def default(cls):
        return cls(year=2000)

    def to_json(self) -> str:
        return json.dumps(dataclasses.asdict(self))

    @classmethod
    def from_json(cls, json_str: str):
        return cls(**json.loads(json_str))

bsm = BotoSesManager(profile_name="awshsh_app_dev_us_east_1", region_name="us-east-1")
bucket = f"{bsm.aws_account_id}-{bsm.aws_region}-data"
s3_client = bsm.s3_client

backend = Backend(bucket=bucket, key="projects/aws_s3_tracker/tracker.json", tracker_class=Tracker)

In [6]:
# read tracker, if the tracker does not exist, it will create a default one
tracker = backend.read(s3_client)
tracker

Tracker(year=2000)

In [9]:
# update tracker and write it back to S3
tracker.year = 2001
backend.write(s3_client, tracker)

# read tracker again, you will get the updated one
tracker = backend.read(s3_client)
tracker

Tracker(year=2001)